In [1]:
import requests

url = "https://eur.osiris-student.nl/student/osiris/student/cursussen/zoeken"

# Your headers here...
headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9,az;q=0.8",
    "Authorization": "undefined undefined",
    "Dnt": "1",
    "Manifest": "23.46_B253_86778e16",
    "Namens_studentnummer": "",
    "Origin": "https://eur.osiris-student.nl",
    "Pragma": "no-cache",
    "Referer": "https://eur.osiris-student.nl/onderwijscatalogus/extern/cursussen",
    "Release_version": "86778e16b9c920e6044e0458fb619ca45ff3300e",
    "Sec-Ch-Ua": "\"Google Chrome\";v=\"119\", \"Chromium\";v=\"119\", \"Not?A_Brand\";v=\"24\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "Taal": "NL",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
}

payload = {
  "from": 0,
  "size": 5000,
  "sort": [
    {"cursus_lange_naam.raw": {"order": "asc"}},
    {"cursus": {"order": "asc"}},
    {"collegejaar": {"order": "desc"}}
  ],
  "aggs": {
    "agg_terms_collegejaar": {
      "filter": {"bool": {"must": []}},
      "aggs": {
        "agg_collegejaar_buckets": {
          "terms": {"field": "collegejaar", "size": 2500, "order": {"_term": "desc"}}
        }
      }
    },
    "agg_terms_blokken_nested.periode_omschrijving": {
      "filter": {"bool": {"must": [{"terms": {"collegejaar": [2018]}}]}},
      "aggs": {
        "agg_blokken_nested.periode_omschrijving": {
          "terms": {
            "field": "blokken_nested.periode_omschrijving",
            "size": 2500,
            "order": {"_term": "asc"},
            "exclude": "Periode: [0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9]"
          }
        },
        "nested_aggs": {
          "nested": {"path": "blokken_nested"},
          "aggs": {
            "nested_aggs": {
              "filter": {"bool": {"must": []}},
              "aggs": {
                "agg_blokken_nested.periode_omschrijving_buckets": {
                  "terms": {
                    "field": "blokken_nested.periode_omschrijving",
                    "size": 2500,
                    "order": {"_term": "asc"},
                    "exclude": "Periode: [0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9]"
                  },
                  "aggs": {"items": {"reverse_nested": {}}}
                }
              }
            }
          }
        }
      }
    },
    "agg_terms_faculteit_naam": {
      "filter": {"bool": {"must": [{"terms": {"collegejaar": [2018]}}]}},
      "aggs": {
        "agg_faculteit_naam_buckets": {
          "terms": {"field": "faculteit_naam", "size": 2500, "order": {"_term": "asc"}}
        }
      }
    },
    "agg_terms_voertalen.voertaal_omschrijving": {
      "filter": {"bool": {"must": [{"terms": {"collegejaar": [2018]}}]}},
      "aggs": {
        "agg_voertalen.voertaal_omschrijving_buckets": {
          "terms": {"field": "voertalen.voertaal_omschrijving", "size": 2500, "order": {"_term": "asc"}}
        }
      }
    },
    "agg_terms_docenten.docent": {
      "filter": {"bool": {"must": [{"terms": {"collegejaar": [2018]}}]}},
      "aggs": {
        "agg_docenten.docent_buckets": {
          "terms": {"field": "docenten.docent", "size": 2500, "order": {"_term": "asc"}}
        }
      }
    }
  },
  "post_filter": {"bool": {"must": [{"terms": {"collegejaar": [2018]}}]}},
  "query": {"bool": {"must": []}}
}


# List to store course IDs
course_IDs = []

# Loop through years
for year in range(2010, 2025):
    # Update the payload for the current year
    payload["aggs"]["agg_terms_blokken_nested.periode_omschrijving"]["filter"]["bool"]["must"][0]["terms"]["collegejaar"] = [year]
    payload["aggs"]["agg_terms_faculteit_naam"]["filter"]["bool"]["must"][0]["terms"]["collegejaar"] = [year]
    payload["aggs"]["agg_terms_voertalen.voertaal_omschrijving"]["filter"]["bool"]["must"][0]["terms"]["collegejaar"] = [year]
    payload["aggs"]["agg_terms_docenten.docent"]["filter"]["bool"]["must"][0]["terms"]["collegejaar"] = [year]
    payload["post_filter"]["bool"]["must"][0]["terms"]["collegejaar"] = [year]

    # Print current year
    print(f"Year: {year}")

    # Make the POST request
    response = requests.post(url, json=payload, headers=headers)

    # Print response status code
    print(f"Response Status Code: {response.status_code}")

    # Check if the request was successful
    if response.status_code == 200:
        # Loop through courses in the response and collect course IDs
        for course in response.json()['hits']['hits']:
            course_IDs.append(course['_source']['id_cursus'])

print("Final Course IDs:", course_IDs)


Year: 2010
Response Status Code: 200
Year: 2011
Response Status Code: 200
Year: 2012
Response Status Code: 200
Year: 2013
Response Status Code: 200
Year: 2014
Response Status Code: 200
Year: 2015
Response Status Code: 200
Year: 2016
Response Status Code: 200
Year: 2017
Response Status Code: 200
Year: 2018
Response Status Code: 200
Year: 2019
Response Status Code: 200
Year: 2020
Response Status Code: 200
Year: 2021
Response Status Code: 200
Year: 2022
Response Status Code: 200
Year: 2023
Response Status Code: 200
Year: 2024
Response Status Code: 200
Final Course IDs: [82374, 84452, 80491, 80220, 82545, 83876, 80855, 84111, 78615, 81497, 81611, 81336, 82554, 86537, 81055, 79130, 81842, 83009, 79258, 86891, 80337, 82556, 80055, 79129, 82608, 83279, 83324, 83690, 83326, 83328, 83342, 83348, 79013, 79257, 83671, 80805, 79127, 79128, 80149, 79341, 79400, 80863, 79998, 82627, 81609, 77490, 77394, 78835, 79145, 79171, 76497, 79397, 79377, 80150, 76492, 79405, 79333, 84245, 79359, 79271, 80956,

In [2]:
len(course_IDs)

18616

In [26]:
with open("course_IDs.txt", "w") as f:
    for course_ID in course_IDs:
        f.write(f"{course_ID}\n")
        